## 제목 : Pytorch Classification NN
## 작성자 : 김수
## 작성 일자 : 2022.02.03

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
import torch.nn.init as init
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
torch.manual_seed(777)
num_epoch = 1000
Learning_Rate = 0.01

In [2]:
training_data = np.loadtxt("G:/공유 드라이브/연구실 폴더/분류대회/Training_data.csv",delimiter=",")
test_data = np.loadtxt("G:/공유 드라이브/연구실 폴더/분류대회/Test_Input.csv",delimiter=",")

In [3]:
cut_line = int(training_data.shape[0]*0.7)
x = training_data[:cut_line,:12]
y = training_data[:cut_line,-4:]
val_x = training_data[cut_line:,:12]
val_y = training_data[cut_line:,-4:]

test_x = test_data[:,:12]

X = torch.tensor(x,dtype=torch.float32)
Y = torch.reshape(torch.tensor(y,dtype=torch.float32),[y.shape[0],y.shape[1]])
Y_not_one_hot_encoding = torch.argmax(Y,dim=1)

Val_X = torch.tensor(val_x,dtype=torch.float32)
Val_Y = torch.reshape(torch.tensor(val_y,dtype=torch.float32),[val_y.shape[0],val_y.shape[1]])
Val_Y_not_one_hot_encoding = torch.argmax(Val_Y,dim=1)

TEST = torch.tensor(test_x,dtype=torch.float32)

print(X.shape)
print(Y.shape)
print(Val_X.shape)
print(Val_Y.shape)
print(TEST.shape)

torch.Size([98000, 12])
torch.Size([98000, 4])
torch.Size([42000, 12])
torch.Size([42000, 4])
torch.Size([60000, 12])
torch.Size([98000, 12])
torch.Size([98000, 4])
torch.Size([42000, 12])
torch.Size([42000, 4])
torch.Size([60000, 12])


In [4]:
# Binary Crossentropy 손실 = loss(x,y) = BCE
# Binary Crossentropy 손실 = loss(x,y) = BCE
model = nn.Sequential(nn.Linear(X.shape[1], 50),nn.ReLU(),nn.Dropout(p=0.3),
                      nn.Linear(50, 75), nn.ReLU(), nn.Dropout(p=0.3),
                      nn.Linear(75, 100), nn.ReLU(), nn.Dropout(p=0.3),
                      nn.Linear(100, 150), nn.ReLU(), nn.Dropout(p=0.3),
                      nn.Linear(150, 200), nn.ReLU(), nn.Dropout(p=0.3),
                      nn.Linear(200, 150), nn.ReLU(), nn.Dropout(p=0.3),
                      nn.Linear(150, 100), nn.ReLU(), nn.Dropout(p=0.3),
                      nn.Linear(100, 50), nn.ReLU(), nn.Dropout(p=0.3),
                      nn.Linear(50, 25), nn.ReLU(), nn.Dropout(p=0.3),
                      nn.Linear(25, Y.shape[1]), nn.Softmax())
optimizer = optim.Adam(model.parameters() ,lr = Learning_Rate)
Loss_fun = torch.nn.CrossEntropyLoss()
Loss = np.zeros((num_epoch,1))
Val_Loss = np.zeros((num_epoch,1))
Accuracy = np.zeros((num_epoch,1))
Val_Accuracy = np.zeros((num_epoch,1))
#학습
for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(X)
    loss = Loss_fun(output,Y_not_one_hot_encoding)
    loss.backward()
    optimizer.step()
    # Accuracy Calculate
    output_not_one_hot_encoding=torch.argmax(output,dim=1)
    accuracy = torch.divide(torch.count_nonzero(torch.eq(output_not_one_hot_encoding,
                                                         Y_not_one_hot_encoding)),Y_not_one_hot_encoding.shape[0])
    with torch.no_grad():
        val_output = model(Val_X)
        val_loss = Loss_fun(val_output,Val_Y)
        val_output_not_one_hot_encoding=torch.argmax(val_output,dim=1)
        val_accuracy = torch.divide(torch.count_nonzero(torch.eq(val_output_not_one_hot_encoding,
                                                         Val_Y_not_one_hot_encoding)),Val_Y_not_one_hot_encoding.shape[0])
    Loss[i,0] = loss.data
    Accuracy[i,0] = accuracy
    Val_Loss[i,0] = val_loss.data
    Val_Accuracy[i,0] = val_accuracy
    if i % 10 == 0:
        print(i,"Loss : ",loss.data,"Accuracy : ",accuracy.data,"Val Loss : ",val_loss.data,"Val Accuracy : ",val_accuracy.data)
        

C:\Users\user\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
C:\Users\user\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0 Loss :  0 Loss :  tensor(1.3865) Accuracy :  tensor(0.2523) Val Loss :  tensor(1.3839) Val Accuracy :  tensor(0.2628)
tensor(1.3865) Accuracy :  tensor(0.2523) Val Loss :  tensor(1.3839) Val Accuracy :  tensor(0.2628)
10 Loss :  tensor(1.2361) Accuracy :  tensor(0.5063) Val Loss :  tensor(1.2032) Val Accuracy :  tensor(0.5153)
10 Loss :  tensor(1.2361) Accuracy :  tensor(0.5063) Val Loss :  tensor(1.2032) Val Accuracy :  tensor(0.5153)
20 Loss :  tensor(1.1371) Accuracy :  tensor(0.5822) Val Loss :  tensor(1.1301) Val Accuracy :  tensor(0.6100)
20 Loss :  tensor(1.1371) Accuracy :  tensor(0.5822) Val Loss :  tensor(1.1301) Val Accuracy :  tensor(0.6100)
30 Loss :  tensor(1.1029) Accuracy :  tensor(0.6821) Val Loss :  tensor(1.0954) Val Accuracy :  tensor(0.6739)
30 Loss :  tensor(1.1029) Accuracy :  tensor(0.6821) Val Loss :  tensor(1.0954) Val Accuracy :  tensor(0.6739)
40 Loss :  tensor(1.0750) Accuracy :  tensor(0.7154) Val Loss :  tensor(1.0781) Val Accuracy :  tensor(0.7147)
40 

KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
print(torch.__version__)

In [ ]:
Predict = model(TEST)
Predict_not_one_hot_encoding = torch.argmax(Predict,dim=1)
print(Predict_not_one_hot_encoding)

In [ ]:
np.savetxt("G:/내 드라이브/연구실/PyTorch/5주차/Test_Predict.csv",Predict_not_one_hot_encoding,delimiter=",",fmt='%d')